
# Pydantic — A Simple Tutorial

## 1. What is Pydantic?

**Pydantic** is a Python library for **data validation and parsing** using **type hints**.

In short:

> You define *what your data should look like*, and Pydantic:
>
> * validates incoming data
> * converts types automatically
> * raises clear errors if data is invalid

This is extremely useful for:

* APIs (FastAPI uses Pydantic heavily)
* Configuration files
* Parsing JSON or external data
* Strongly-typed Python code

---

## 2. The Problem Pydantic Solves

### Without Pydantic (manual validation)

```python
def create_user(data: dict):
    if not isinstance(data["id"], int):
        raise ValueError("id must be int")

    if not isinstance(data["name"], str):
        raise ValueError("name must be str")

    if not isinstance(data["age"], int):
        raise ValueError("age must be int")

    return data
```

Problems:

* Verbose
* Error-prone
* No automatic type conversion
* Hard to scale

---

## 4. Your First Pydantic Model

### Basic example

```python
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str
    age: int
```

Now you have a **schema** for your data.

---

## 5. Creating an Instance (Validation Happens Here)

```python
user = User(
    id=1,
    name="Alice",
    age=30
)
```

If the data is valid → object is created.

### Automatic type conversion

```python
user = User(
    id="1",
    name="Alice",
    age="30"
)

print(user.id, type(user.id))
print(user.age, type(user.age))
```

Output:

```
1 <class 'int'>
30 <class 'int'>
```

Pydantic **parsed** strings into integers automatically.

---

## 6. What Happens on Invalid Data?

```python
User(
    id="abc",
    name="Alice",
    age=30
)
```

Error:

```text
ValidationError: value is not a valid integer
```

Pydantic raises **structured, human-readable errors**.

---

## 7. Accessing Data

Pydantic models behave like **typed objects**, not dictionaries.

```python
print(user.id)
print(user.name)
print(user.age)
```

You can also convert back to dict:

```python
user_dict = user.model_dump()
```

---

## 8. Optional Fields and Default Values

### Optional fields

```python
from typing import Optional

class User(BaseModel):
    id: int
    name: str
    age: Optional[int] = None
```

Now `age` can be missing.

---

### Default values

```python
class User(BaseModel):
    id: int
    name: str
    is_active: bool = True
```

---

## 9. Field Validation Rules

You can add **constraints** using `Field`.

```python
from pydantic import BaseModel, Field

class User(BaseModel):
    id: int
    name: str
    age: int = Field(ge=0, le=120)
```

Meaning:

* `ge=0` → greater or equal to 0
* `le=120` → less or equal to 120

---

## 10. Custom Validators

When built-in constraints are not enough.

```python
from pydantic import BaseModel, field_validator

class User(BaseModel):
    name: str

    @field_validator("name")
    @classmethod
    def name_must_not_be_empty(cls, value):
        if not value.strip():
            raise ValueError("name must not be empty")
        return value
```

---

## 11. Nested Models

Pydantic handles **nested structures** naturally.

```python
class Address(BaseModel):
    city: str
    zip_code: str

class User(BaseModel):
    id: int
    name: str
    address: Address
```

Usage:

```python
user = User(
    id=1,
    name="Alice",
    address={
        "city": "Lausanne",
        "zip_code": "1000"
    }
)
```

Pydantic automatically creates the nested `Address` object.

---

## 12. Parsing JSON

```python
json_data = """
{
    "id": 1,
    "name": "Alice",
    "age": 30
}
"""

user = User.model_validate_json(json_data)
```

---
